In [ ]:
import os
from contextlib import suppress

import open3d as o3d
import numpy as np
from scipy.spatial.transform import Rotation

In [ ]:
path = 'edf_env/edf_env/assets/mug_task/mugs/test/mug2'
in_path = os.path.join(path, 'mug.ply')
out_path = os.path.join(path, 'mug_simplified.stl')

detail = 32
reorient = reorient = Rotation.from_euler('XYZ', [0, 0, np.pi]) * Rotation.from_euler('XYZ', [np.pi/2, 0, 0])
reorient = reorient.as_matrix() # handle to y-axis, and z-axis is upward.
rescale = 0.08
recenter = True

print(reorient)

In [ ]:
# if in_path.split('.')[-1] == 'dae':
#     import aspose.threed as a3d

#     with suppress(RuntimeError):
#     scene = a3d.Scene.from_file(in_path)

#     path_str = ''
#     for _str in in_path.split('.')[:-1] + ['.ply']:
#         path_str += _str
    
#     scene.save(path_str)

In [ ]:
mesh_in = o3d.io.read_triangle_mesh(in_path)
mesh_in.compute_vertex_normals()
print(
    f'Input mesh has {len(mesh_in.vertices)} vertices and {len(mesh_in.triangles)} triangles'
)

In [ ]:
#o3d.visualization.draw_geometries([mesh_in])

In [ ]:
assert detail >= 1.
voxel_size = max(mesh_in.get_max_bound() - mesh_in.get_min_bound()) / detail

mesh_smp = mesh_in.simplify_vertex_clustering(
    voxel_size=voxel_size,
    contraction=o3d.geometry.SimplificationContraction.Average)
print(
    f'Simplified mesh has {len(mesh_smp.vertices)} vertices and {len(mesh_smp.triangles)} triangles'
)

In [ ]:
if reorient is not False:
    mesh_smp = mesh_smp.rotate(reorient, center=(0, 0, 0))

if rescale is not False:
    vertices = np.asarray(mesh_smp.vertices)
    z_scale = vertices[:,-1].max() - vertices[:,-1].min()
    vertices = vertices / z_scale * rescale
    mesh_smp.vertices = o3d.utility.Vector3dVector(vertices)

if recenter is not False:
    # ##### Vertex Centroid #####
    # # equivalent to mesh_smp.get_center()
    # vertices = np.asarray(mesh_smp.vertices)
    # centroid = vertices.mean(axis=-2)
    # vertices = vertices - centroid
    # mesh_smp.vertices = o3d.utility.Vector3dVector(vertices)

    ##### Mesh Center of Mass #####
    vertices = mesh_smp.sample_points_uniformly(number_of_points=5000)
    centroid = np.asarray(vertices.points).mean(axis=-2)
    vertices = np.asarray(mesh_smp.vertices) - centroid
    mesh_smp.vertices = o3d.utility.Vector3dVector(vertices)

In [ ]:
scale = rescale if rescale else z_scale
coord_axis = o3d.geometry.TriangleMesh.create_coordinate_frame()
coord_axis = coord_axis.scale(scale=scale, center=[0, 0, 0])


mesh_smp.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh_smp, coord_axis])

In [ ]:
print(f"z-axis rescaled from {z_scale} to {scale}")
print(f"Centroid: {np.asarray(mesh_smp.vertices).mean(axis=-2)}")

In [ ]:
o3d.io.write_triangle_mesh(out_path, mesh_smp)